In [1]:
import subprocess
import os

def check_ffmpeg_and_ffprobe():
    try:
        # Check ffmpeg
        subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True)
        # Check ffprobe 
        subprocess.run(['ffprobe', '-version'], capture_output=True, check=True)
        return True
    except (subprocess.SubprocessError, FileNotFoundError):
        print("FFmpeg and/or FFprobe are not installed or not in PATH. Please run ./setup.sh from the project root to install them.\n"
              "Alternatively, you can install manually:\n"
              "- On macOS: brew install ffmpeg\n"
              "- On Ubuntu/Debian: sudo apt install ffmpeg\n"
              "- On Windows: Download from https://ffmpeg.org/download.html")
        
        return False

# Run the function and print result
is_ffmpeg_installed = check_ffmpeg_and_ffprobe()
print(f"FFmpeg and FFprobe are installed: {is_ffmpeg_installed}")


FFmpeg and FFprobe are installed: True


In [2]:
import yt_dlp

def download_youtube_audio(url, output_path=None):
    """
    Download audio from a YouTube video or playlist
    
    Args:
        url (str): YouTube video or playlist URL
        output_path (str, optional): Path where the audio should be saved.
            For single videos: filename.mp3
            For playlists: playlist_name/video_title.mp3
            
    Returns:
        str: Path to the directory where files were downloaded
    """
    # First check if ffmpeg and ffprobe are installed
    if not check_ffmpeg_and_ffprobe():
        return None
        
    # Get ffmpeg location
    try:
        ffmpeg_path = subprocess.check_output(['which', 'ffmpeg']).decode().strip()
        ffmpeg_dir = os.path.dirname(ffmpeg_path)
    except subprocess.SubprocessError:
        print("Could not locate ffmpeg. Please ensure it's installed and in your PATH")
        return None

    # Create downloads directory if it doesn't exist
    downloads_dir = os.path.join(os.path.dirname(os.getcwd()), 'downloads')
    os.makedirs(downloads_dir, exist_ok=True)

    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'ffmpeg_location': ffmpeg_dir,  # Add ffmpeg directory location
        'quiet': False,
        'no_warnings': False,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # Extract info first to determine if it's a playlist
            info = ydl.extract_info(url, download=False)
            
            if 'entries' in info:  # It's a playlist
                # Create playlist directory
                playlist_name = info.get('title', 'playlist')
                playlist_dir = os.path.join(downloads_dir, playlist_name)
                os.makedirs(playlist_dir, exist_ok=True)
                
                # Set output template for playlist items
                ydl_opts['outtmpl'] = os.path.join(playlist_dir, '%(title)s.%(ext)s')
                final_dir = playlist_dir
            else:  # Single video
                if output_path:
                    ydl_opts['outtmpl'] = os.path.join(downloads_dir, output_path)
                else:
                    ydl_opts['outtmpl'] = os.path.join(downloads_dir, '%(title)s.%(ext)s')
                final_dir = downloads_dir
            
            # Download with updated options
            with yt_dlp.YoutubeDL(ydl_opts) as ydl_download:
                ydl_download.download([url])
                
        print("Audio download completed successfully!")
        return final_dir
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def combine_audio_files(input_dir, output_filename='combined_audio.mp3'):
    """
    Combines all MP3 files in the input directory into a single audio file.
    
    Args:
        input_dir (str): Directory containing the MP3 files
        output_filename (str): Name of the output file (default: 'combined_audio.mp3')
    
    Returns:
        str: Path to the combined audio file if successful, None otherwise
    """
    # First check if ffmpeg is installed
    if not check_ffmpeg_and_ffprobe():
        return None
        
    try:
        # Get all MP3 files in the directory
        mp3_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.mp3')])
        
        if not mp3_files:
            print(f"No MP3 files found in {input_dir}")
            return None
            
        print(f"Found {len(mp3_files)} MP3 files to combine")
            
        # Create a temporary file listing all input files
        concat_file = os.path.join(input_dir, 'concat_list.txt')
        with open(concat_file, 'w', encoding='utf-8') as f:
            for mp3_file in mp3_files:
                # Escape special characters in file path
                safe_path = os.path.join(input_dir, mp3_file).replace("'", "'\\''")
                f.write(f"file '{safe_path}'\n")
        
        # Output path
        output_path = os.path.join(input_dir, output_filename)
        
        # Combine audio files using ffmpeg
        cmd = [
            'ffmpeg',
            '-f', 'concat',
            '-safe', '0',
            '-i', concat_file,
            '-c', 'copy',
            output_path
        ]
        
        print("Combining audio files...")
        # Run the command
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode != 0:
            print(f"Error combining files: {result.stderr}")
            return None
        
        # Clean up the temporary file
        os.remove(concat_file)
        
        print(f"Successfully combined {len(mp3_files)} audio files into {output_path}")
        return output_path
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def process_playlist(playlist_url, output_filename='combined_audio.mp3'):
    """
    Downloads a YouTube playlist and combines all videos into a single audio file.
    
    Args:
        playlist_url (str): URL of the YouTube playlist
        output_filename (str): Name of the output file (default: 'combined_audio.mp3')
    
    Returns:
        str: Path to the combined audio file if successful, None otherwise
    """
    # Download the playlist
    downloads_dir = download_youtube_audio(playlist_url)
    if not downloads_dir:
        print("Failed to download playlist")
        return None
    
    # Combine the audio files
    return combine_audio_files(downloads_dir, output_filename)


In [8]:

# Example usage

# Can be video url or playlist url
video_url = "https://www.youtube.com/watch?v=rFZrL1RiuVI"
combined_audio_path = process_playlist(video_url, "peter_thiel_interview.mp3")

if combined_audio_path:
    print(f"Final combined audio file is at: {combined_audio_path}")

# If you want to specify an output path (for single videos only):
# download_youtube_audio("https://www.youtube.com/watch?v=example", "my_audio.mp3")

[youtube] Extracting URL: https://www.youtube.com/watch?v=rFZrL1RiuVI
[youtube] rFZrL1RiuVI: Downloading webpage
[youtube] rFZrL1RiuVI: Downloading ios player API JSON
[youtube] rFZrL1RiuVI: Downloading mweb player API JSON
[youtube] rFZrL1RiuVI: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=rFZrL1RiuVI
[youtube] rFZrL1RiuVI: Downloading webpage
[youtube] rFZrL1RiuVI: Downloading ios player API JSON
[youtube] rFZrL1RiuVI: Downloading mweb player API JSON
[youtube] rFZrL1RiuVI: Downloading m3u8 information
[info] rFZrL1RiuVI: Downloading 1 format(s): 251
[download] Destination: /Users/suchintan/Development/audiobook-generator/downloads/Peter Thiel： Going from Zero to One.webm
[download] 100% of   12.43MiB in 00:00:01 at 6.87MiB/s     
[ExtractAudio] Destination: /Users/suchintan/Development/audiobook-generator/downloads/Peter Thiel： Going from Zero to One.mp3
Deleting original file /Users/suchintan/Development/audiobook-generator/downloads/Peter 

In [4]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
from pprint import pprint
pprint(TTS().list_models().list_models())

['tts_models/multilingual/multi-dataset/xtts_v2',
 'tts_models/multilingual/multi-dataset/xtts_v1.1',
 'tts_models/multilingual/multi-dataset/your_tts',
 'tts_models/multilingual/multi-dataset/bark',
 'tts_models/bg/cv/vits',
 'tts_models/cs/cv/vits',
 'tts_models/da/cv/vits',
 'tts_models/et/cv/vits',
 'tts_models/ga/cv/vits',
 'tts_models/en/ek1/tacotron2',
 'tts_models/en/ljspeech/tacotron2-DDC',
 'tts_models/en/ljspeech/tacotron2-DDC_ph',
 'tts_models/en/ljspeech/glow-tts',
 'tts_models/en/ljspeech/speedy-speech',
 'tts_models/en/ljspeech/tacotron2-DCA',
 'tts_models/en/ljspeech/vits',
 'tts_models/en/ljspeech/vits--neon',
 'tts_models/en/ljspeech/fast_pitch',
 'tts_models/en/ljspeech/overflow',
 'tts_models/en/ljspeech/neural_hmm',
 'tts_models/en/vctk/vits',
 'tts_models/en/vctk/fast_pitch',
 'tts_models/en/sam/tacotron-DDC',
 'tts_models/en/blizzard2013/capacitron-t2-c50',
 'tts_models/en/blizzard2013/capacitron-t2-c150_v2',
 'tts_models/en/multi-dataset/tortoise-v2',
 'tts_mode

In [9]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Run TTS
tts.tts_to_file(text="Tolstoy opens Anna Karenina by observing: “All happy families are alike; each unhappy family is unhappy in its own way.” Business is the opposite. All happy companies are different: each one earns a monopoly by solving a unique problem. All failed companies are the same: they failed to escape competition.", speaker_wav=combined_audio_path, language="en", file_path="downloads/output.wav")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


/Users/suchintan/Development/audiobook-generator/.venv/lib/python3.11/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self

 > Text splitted to sentences.
['Tolstoy opens Anna Karenina by observing: “All happy families are alike; each unhappy family is unhappy in its own way.”', 'Business is the opposite.', 'All happy companies are different: each one earns a monopoly by solving a unique problem.', 'All failed companies are the same: they failed to escape competition.']
 > Processing time: 40.66623306274414
 > Real-time factor: 1.6886571182232817


'output.wav'